# Importing pakages

In [1]:
import wandb
import pickle
import time
from src.datahandle import *
import time
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


# Setting up hyperparams

In [2]:
#hyper
directory = '../../extras/data/A_training_given/training_data/'
training_folder = 'training_data'
training_labels_file= 'training_norm.csv'
#
collected_directory = '../../extras/data/Z_collected_raw/' 
#
quiz_directory = '../../extras/data/C_testing_given/test_data/'
quiz_training_folder = 'test_data'
#configurations
conf_tracking = 1
seed = 0
data_shape = 'original'
parameter_init_scale = 0.01
split= 0.8
batch_size = 256
n_epochs = 8
lr = 0.001

# Setting up DataLoaders

In [3]:
#dataloading object
training_object= DataLoader(
        directory,
        training_folder,
        training_labels_file
        )
collected_object = DataLoader(
        collected_directory        
        )

# Loading data information (not loading images yet)

In [4]:
collected_train, collected_test = collected_object.LoadCollectedData_info(
        split=split,
        batch_size=batch_size #for tf this batch size remains as 1
        )
#train test split
train,test = training_object.LoadModelData_info(
        split = split, 
        batch_size =batch_size)#for tf this batch size remains as 1

In [5]:
collected_train[0][1],train[1][1:10]

(array(['../../extras/data/Z_collected_raw/1648217084653_90_35.png',
        '1648217084653', '0.571', '0.899'], dtype='<U58'),
 array([['../../extras/data/A_training_given/training_data/training_data/11705.png',
         '11705', '0.625', '0.885'],
        ['../../extras/data/A_training_given/training_data/training_data/7262.png',
         '7262', '0.6875', '0.052'],
        ['../../extras/data/A_training_given/training_data/training_data/6443.png',
         '6443', '0.75', '0.965'],
        ['../../extras/data/A_training_given/training_data/training_data/6877.png',
         '6877', '0.75', '0.886'],
        ['../../extras/data/A_training_given/training_data/training_data/5105.png',
         '5105', '0.5', '0.946'],
        ['../../extras/data/A_training_given/training_data/training_data/2186.png',
         '2186', '0.6875', '0.929'],
        ['../../extras/data/A_training_given/training_data/training_data/10661.png',
         '10661', '0.75', '0.893'],
        ['../../extras/data/A_t

# Stacking collected and kaggle given data

In [6]:
train_XY = train #np.vstack((train,collected_train))
test_XY= test[0] #np.vstack((test[0],collected_test[0]))

# Checking DataLoading functionality 

In [7]:
for i in range(1):
    test_batch_X,test_batch_Y = training_object.Load_batch(train_XY[i], data_shape=data_shape)
image_shape = test_batch_X.shape[1:]
image_shape

(240, 320, 3)

# Setting up data augmentation

In [8]:
data_augmentation = keras.Sequential(
  [
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.RandomZoom(0.2),
        #layers.experimental.preprocessing.RandomContrast(0.2),
  ]
)

# Setting up preprocessing for MobilenetV2

# Transfer Learning model

## to do
- [x] Seperate TF_cnn from TF_transfer
- [x] Remove rescaling in datahandling step (Now done by the preprocess layer)
- [x] Correctly rescale collected images labels (speed and angle)
- [x] Soften output
- [ ] Train the transfer learning model
- [ ] Tune the transfer learning model

In [9]:
preprocess_input = tf.keras.applications.efficientnet_v2.preprocess_input

In [10]:
Base = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(input_shape=image_shape,include_top=False,weights='imagenet')
Base.trainable = False

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
inference_layer = tf.keras.layers.Dense(2)

In [12]:
example_batch = training_object.Load_batch(train_XY[0], data_shape=data_shape)
preprocess_input(example_batch[0][0]).shape

(240, 320, 3)

In [13]:
inputs = tf.keras.layers.Input(shape=image_shape)
x = preprocess_input(inputs)
x = Base(inputs)
x = global_average_layer(x)
outputs = inference_layer(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 8, 10, 1280)      5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 5,921,874
Trainable params: 2,562
Non-trainable params: 5,919,312
_________________________________________________________________


# Weights and Biases configuration

In [15]:
#wandb tracking
if conf_tracking:
    config = {
     "model_type" : 'Transfer Learning with MobileNetV2',
     "model_shape" : str(model.layers),
     "learning_rate": lr,
     "data_split": split,
     "batch_size": batch_size,
     "epochs": n_epochs,
    }
    run = wandb.init(project="Autonomous-driving", entity="mo379",config=config)

wandb: Currently logged in as: mo379 (use `wandb login --relogin` to force relogin)


# Training Loop (With wandb logging)

In [16]:
model_path = 'pkls/tf_cnn_augmented.pkl'
#model = tf.keras.models.load_model(model_path, custom_objects=None, compile=True, options=None)
test_batches = test_XY[0:2750].reshape(10,-1,4)
for _ in range(n_epochs):
    for i in range(len(train_XY)):
        X,Y = training_object.Load_batch(train_XY[i], data_shape=data_shape)
        train_metrics = model.train_on_batch(X,Y, return_dict=True) 
        if i % 3 == 0:
            X_test,Y_test = training_object.Load_batch(test_batches[np.random.randint(0,9)], data_shape=data_shape)
            test_metrics = model.test_on_batch(
                X_test, Y_test, sample_weight=None, reset_metrics=False, return_dict=True
            )
        
            if conf_tracking==1:
                wandb.log({"test_loss": test_metrics['loss']})
                wandb.log({"batch_loss": train_metrics['loss']})



InternalError: Could not synchronize CUDA stream: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered

In [17]:
run.finish()

batch_loss,█▃▃▂▂▁▂▁▁▁
test_loss,█▃▃▂▂▂▂▁▁▁
batch_loss,0.07282
test_loss,0.06789


In [ ]:
tf.keras.models.save_model(
    model,
    model_path,
    overwrite=False,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
    save_traces=True
)

# Loading the Quiz data

In [ ]:
quiz_object= DataLoader(
    quiz_directory,
    quiz_training_folder,
)
quiz_train = quiz_object.LoadQuizData_info()
X,image_order = quiz_object.Load_batch_quiz(quiz_train,data_shape=data_shape)

# Making predictions

In [ ]:
prds = model.predict(X)
final_prd = np.column_stack((image_order,prds))
final_ordered = final_prd[final_prd[:, 0].argsort()]
df = pd.DataFrame(final_ordered, columns = ['image_id','angle','speed'])
df = df.astype({'image_id': 'int32'})
df.to_csv('transfer_MobileNetV3Small.csv', index=False,)